In [3]:
%matplotlib inline  
%load_ext autoreload
%autoreload 2

In [1]:
import numpy as np
import math

In [7]:
from toolz.curried import curry
from toolz.curried import map
from toolz.curried import pipe

In [199]:
fft = curry(np.fft.fft)  # pylint: disable=invalid-name

ifft = curry(np.fft.ifft)  # pylint: disable=invalid-name

fftn = curry(np.fft.fftn)  # pylint: disable=invalid-name

ifftn = curry(np.fft.ifftn)  # pylint: disable=invalid-name

fftshift = curry(np.fft.fftshift)  # pylint: disable=invalid-name

conj = curry(np.conj)

func = curry(lambda x, y: conj(x) * fftn(y))

@curry
def return_slice(x_data, cutoff):
    
    s = np.asarray(x_data.shape).astype(int) // 2 + 1
    
    if x_data.ndim == 2:
        return x_data[(s[0] - cutoff):(s[0] + cutoff), 
                      (s[1] - cutoff):(s[1] + cutoff)]
    elif x_data.ndim ==3:
        return x_data[(s[0] - cutoff):(s[0] + cutoff), 
                      (s[1] - cutoff):(s[1] + cutoff), 
                      (s[2] - cutoff):(s[2] + cutoff)]
    else:
        print('Incorrect Number of Dimensions!')

        
@curry
def corr_master_(x_data, cutoff, func):
    return pipe(x_data,
                fftn,
                func, 
                ifftn, 
                fftshift, 
                return_slice(cutoff=cutoff))


@curry        
def corr_master_auto(cutoff, *args):
    return pipe(args[0],
                fftn,
                lambda x: conj(x)*x, 
                ifftn, 
                fftshift, 
                return_slice(cutoff=cutoff))
    
@curry        
def corr_master_cross(cutoff, *args):
    return pipe(args[0], 
                fftn, 
                lambda x: conj(x) * fftn(args[1]), 
                ifftn, 
                fftshift, 
                return_slice(cutoff = cutoff))

# @curry        
# def corr_master_cross(cutoff, *args):
#     return pipe(args[0], 
#                corr_master_(cutoff=cutoff, func=func(y=args[1])))

    
@curry        
def corr_master(corrtype, cutoff, *args):

    if corrtype == 'auto':
        return corr_master_auto(cutoff, *args) 
    elif corrtype == 'cross': 
        return corr_master_cross(cutoff, *args)
    else:
        print("Please either <Auto> or <Cross> as correlation type")

In [206]:
gg1 = np.load('gg1.npy')
gg2 = np.load('gg2.npy')
np.random.seed(101)
s = (5, 5)
x = (np.random.random(s) > 0.5).astype(int)
np.random.seed(99)
y = (np.random.random(s) > 0.5).astype(int)
gg = abs(corr_master('auto', 3, x))/x.size
gg_ = abs(corr_master('cross', 3, x, y))/x.size
print(np.allclose(gg, gg1))
print(np.allclose(gg_, gg2))

True
True


In [125]:
print(x1)
print(gg1)
print(gg2)

[[1 1 0 0 1]
 [1 0 1 1 0]
 [1 0 0 1 1]
 [0 0 1 1 0]
 [1 1 0 0 0]]
[[ 0.32  0.24  0.24  0.24  0.28]
 [ 0.24  0.2   0.24  0.32  0.36]
 [ 0.16  0.28  0.52  0.28  0.16]
 [ 0.36  0.32  0.24  0.2   0.24]
 [ 0.28  0.24  0.24  0.24  0.32]]
[[ 0.28  0.28  0.32  0.36  0.24]
 [ 0.28  0.28  0.28  0.24  0.36]
 [ 0.32  0.24  0.32  0.32  0.24]
 [ 0.28  0.36  0.2   0.32  0.28]
 [ 0.24  0.36  0.32  0.28  0.28]]


In [204]:
np.save('gg1', gg1)
np.save('gg2', gg2)
np.save()